In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import datetime as dt
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
import intake,param
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

from tqdm import tqdm, trange
import ipywidgets as iw

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr
import xarray.ufuncs as xu

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")    

In [ ]:
#import utils
import hv_utils as  hvu

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

## Set default holoviews style options

In [ ]:
IMG_H = 300; IMG_W = 300
CURVE_H = 200; CURVE_W = 300

In [ ]:
%opts Image [colorbar=True, tools=['hover'], active_tools=['wheel_zoom']] Curve [tools=['hover']]

In [ ]:
opts.defaults(
    opts.WMTS(active_tools=['wheel_zoom'],
              width=IMG_W, height=IMG_H),
    opts.Image(active_tools=['wheel_zoom'], tools=['hover'], colorbar=True,
              width=IMG_W, height=IMG_H),
    opts.Curve(active_tools=['wheel_zoom'], tools=['hover'],
               width=CURVE_W, height=CURVE_H),
    opts.Scatter(active_tools=['wheel_zoom'], tools=['hover'],
                width=CURVE_W, height=CURVE_H),
    opts.HLine(active_tools=['wheel_zoom'], tools=['hover']),
    opts.RGB(active_tools=['wheel_zoom'], tools=['hover'],
             width=IMG_W, height=IMG_H),
    opts.Overlay(active_tools=['wheel_zoom'],
                width=IMG_W, height=IMG_H),
)

img_opts = dict(active_tools=['wheel_zoom'], tools=['hover'], colorbar=True,
              width=IMG_W, height=IMG_H)
curve_opts = dict(active_tools=['wheel_zoom'], tools=['hover'],
               width=CURVE_W, height=CURVE_H)
scatter_opts = dict(active_tools=['wheel_zoom'], tools=['hover'],
                width=CURVE_W, height=CURVE_H)
hline_opts = dict(active_tools=['wheel_zoom'], tools=['hover'])
overlay_opts = dict(active_tools=['wheel_zoom'], width=IMG_W, height=IMG_H)


---
## Load Datasets

In [ ]:
# Southern Africa Dataset

fpath_sa = Path.home()/'data/mint/FLDAS/FLDAS_NOAH01_A_SA_D.001/2019/04/FLDAS_NOAH01_A_SA_D.A201904*.001.nc'
fpath_ea = Path.home()/'data/mint/FLDAS/FLDAS_NOAH01_A_EA_D.001/2019/04/FLDAS_NOAH01_A_EA_D.A201904*.001.nc'
fpath_sa = str(fpath_sa)
fpath_ea = str(fpath_ea)
ds_sa = xr.open_mfdataset(fpath_sa)
ds_sa = ds_sa.drop_dims('bnds')

ds_ea = xr.open_mfdataset(fpath_ea)
ds_ea = ds_ea.drop_dims('bnds')

         
# print(ds_ea)
# print(ds_sa)

In [ ]:
xrd_ea = ds_ea.persist()
xrd_sa = ds_sa.persist()

In [ ]:
# data variable list
varnames_ea = list(ds_ea.data_vars.keys())
varnames_sa = list(ds_sa.data_vars.keys())
varnames = varnames_ea
varname = varnames[3]
print(varname)

# create holoviews dataset containers 
kdims = ['X','Y','time']
hvd_ea = hv.Dataset(xrd_ea, kdims)
hvd_sa = hv.Dataset(xrd_sa, kdims)


In [ ]:
# colormaps
## discretize it conveniently using holoview's "color_level" option
t_fixed = '2019-04-05'
varname = varnames[5] 
print("Selecting a datavariable at a fixed time point: ", t_fixed, varname)

# timg_ea = hvd_ea.select(time=t_fixed).to(gv.Image, kdims=['X', 'Y'], vdims=varname) #this returns a holomap, not a hv.Image object
# To construct an hv.Image object, we need to pass in the xr.DataArray (ie. one value variable)
print(xrd_ea[varname].isel(time=3) )
timg_ea = gv.Image(xrd_ea[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
timg_sa = gv.Image(xrd_sa[varname].isel(time=3) , ['X','Y'], crs=ccrs.PlateCarree()) #Opt: vdims=varname
# print(timg_sa)
# gv.tile_sources.Wikipedia * timg_ea.opts(alpha=0.5,width=W_IMG, height=H_IMG) #+ timg_sa.opts(width=W_IMG, height=H_IMG)

## Basemap tile

We need to handle the projection from latlon to web mercator (which is what the hv.tiles expect).


In [ ]:
wmts_url = 'https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'
basemap = gv.tile_sources.EsriImagery

In [ ]:
# %%opts WMTS [width=IMG_W, height=IMG_H]
# basemap * timg_ea
# gvts.EsriNatGeo*timg_ea

---
## Add Callbacks 

### PolyEdit stream

Draw a polygon with mouse clicks

In [ ]:
from holoviews.streams import *

---
### `hv.Box` constructors
Modified: Jun 20, 2019

In [ ]:
center_xs = np.arange(5)
center_ys = [1,3,6,9,12]
sizes = [1, 1, 1, 2, 2]
df_boxes = pd.DataFrame( {'x': center_xs, 'y': center_ys, 'size': sizes}) 
df_boxes

- `hv.Box(x,y, length)`: square box of length=length, centered at x,y
- `hv.Box(x,y, (w,h))`: box of width=w, height=h, centered at x,y
- `hv.Box(x,y, size, aspect)`: a box of height=size, width=aspect*size, centered at x,y
- `hv.Box(x,y, length, orientation)`: rotate the box by orientation [rad] anti-clockwise

Note: the last argument is actually a keyword argument of `spec`.
For example, `hv.Box(1,1,spec=(1,2))` is the same as `hv.Box(1,1, (1,2))`.

In [ ]:
%opts Box (color='red', line_width=3)
geom_overlay_opts = dict(show_grid=True, padding=0.2, aspect='equal',show_legend=True )

In [ ]:
(
    hv.Box( 0,0,2 )
    * hv.Box(0,0,1).opts(color='blue') 
    * hv.Box(0,0,1, aspect=1.5).opts(color='lightblue')

    * hv.Box(0,0,0.5, orientation=np.pi/6).opts(color='green')
).opts(**geom_overlay_opts)

---
## hv.Ellipse constructor
Creates a circle or ellipse

 The constructors look very similar to hv.Box constructor
- `hv.Ellipse(x,y, size)`: a circle of diameter=size centered at x,y. (ie. its radius=size/2)
- `hv.Ellipse(x,y, (width_xaxis, width_yaxis) )`: an ellipse with xaxis of length=width_xaxis 
        and yaxis of length=width_yaxis
- `hv.Ellipse(x,y,size, aspect)`: an ellipse with yaxis_width=size, width_xaxis=aspect*size

Note: the last argument is actually a keyword argument of `spec`.
For example, `hv.Ellipse(x,y,size)` is equivalent to `hv.Ellipse(x,y, spec=size`.


In [ ]:
%opts Ellipse (line_width=3)'

In [ ]:
(
    hv.Ellipse( 0,1,0.5).opts(color='black')
    * hv.Ellipse(0,1,0.5, aspect=2).opts(color='red')
    * hv.Ellipse(0,1, (2,1), label='ellipse (2,1)').opts(color='blue')
    * hv.Ellipse(0,1, (2,1), orientation=np.pi/3).opts(color='green')
).opts(**geom_overlay_opts)

## hv.Polygon constructor
Create a polygon 
- from data array: numpy array, geopandas (with `geom` column that stores shapely shapes)
- from a file (eg.shapefile, geojson?)

Polygons = a contiguous filled area in a 2D space as a list of polygon geometries
- accepts  a list of arrays, dataframes, a dictionary of columns, etc
- supports `holes` key to represent empty interior regions

In [ ]:
data = {('x','y'): [ (0., 1., 

In [ ]:
path = [(0,0), (0,2), (2,3), (3,0)]
path2 = [ (1,0), (0,1), (2,2), (2,1) ]

paths = [path, path2]
vals = [0.1, 0.9]
data = dict(

In [ ]:
hv.Polygons(paths, 

---
## hv.Polygon constructor
- hv.Polygon([data_dict1, data_dict2, ...]) where each `data_dict` is a dictionary representation of a shape item (like JSON-format). Each `data_dict` has key,value of
    - `('x','y') -> shape data, eg:`hv.Box(0.,0.,2).array()

In [ ]:
hv.Box(0,0,2).array()

In [ ]:
shape1 = {('x','y'): hv.Box(0.5,0.5,1).array(),
          'z': 1}
shape2 = {('x','y'): hv.Box(3,1.5, (2,1)).array(),
          'z': 2}
shape3 = {('x','y'): hv.Ellipse(5,5,2).array(),
          'z':3}

poly_data = [shape1, shape2, shape3]

In [ ]:
polys = hv.Polygons(poly_data, vdims='z')

In [ ]:
# Define a subscriber to the polyedit stream
from ipywidgets import Output
out = Output()


In [ ]:
@out.capture(clear_output=True)
def listener(*args, **kwargs):
    print(args)
    print(kwargs)
    

In [ ]:
# Try if the stream works
# poly_opts = opts.Polygons(active_tools=['poly_edit'], fill_alpha=0.7)
poly_opts = opts.Polygons(active_tools=['poly_draw'], fill_alpha=0.7)

# Add the polyedit stream to hv.Polygons
# shared=True allows the same tool to be used to edit multiple polygon objects
poly_edit_kws = dict(vertex_style={'color':'red'},
                    shared=True)
poly_edit = PolyEdit(source=polys, **poly_edit_kws)
poly_edit.add_subscriber(listener)


# Add the polydraw stream to hv.Polygons
polydraw = PolyDraw(source=polys)

In [ ]:
polys.opts(active_tools=['poly_draw'])


In [ ]:
path = hv.Path([[(1, 5), (9, 5)]])
poly = hv.Polygons([[(2, 2), (5, 8), (8, 2)]])
path_stream = streams.PolyDraw(source=path, drag=True, show_vertices=True)
poly_stream = streams.PolyDraw(source=poly, drag=True, num_objects=4,
                               show_vertices=True, styles={
                                   'fill_color': ['red', 'green', 'blue']
                               })

(path * poly).opts(
    opts.Path(color='red', height=400, line_width=5, width=400),
    opts.Polygons(fill_alpha=0.3, active_tools=['poly_draw']))

In [ ]:
path_stream.element


In [ ]:
poly_stream.element

---
Pointdraw

In [ ]:
data = ([0, 0.5, 1], [0, 0.5, 0], ['red', 'green', 'blue'])

points = hv.Points(data, vdims='color').redim.range(x=(-.1, 1.1), y=(-.1, 1.1))
points.columns()

---
## Bounds
Modified: Jun 22, 2019


In [ ]:
from holoviews.streams import *

In [ ]:
box_stream = BoundsXY(name='box', source = timg_ea)

In [ ]:
timg_ea.opts(**img_opts)

---


In [ ]:
fpath = '/Users/hayley/Playground/Pyvizsuite/holoviews/examples/assets/twophoton.npz'
data = np.load(fpath)
calcium_array = data['Calcium']

In [ ]:
ds = hv.Dataset((np.arange(50), np.arange(111), np.arange(62), calcium_array),
                ['Time', 'x', 'y'], 'Fluorescence')

In [ ]:
polys = hv.Polygons([])
box_stream = streams.BoxEdit(source=polys)

def roi_curves(data):
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([], 'Time', 'Fluorescence')})
    
    curves = {}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0, x1, y0, y1) in enumerate(data):
        selection = ds.select(x=(x0, x1), y=(y0, y1))
        curves[i] = hv.Curve(selection.aggregate('Time', np.mean))
    return hv.NdOverlay(curves)

hlines = hv.HoloMap({i: hv.VLine(i) for i in range(50)}, 'Time')
dmap = hv.DynamicMap(roi_curves, streams=[box_stream])

In [ ]:
im = ds.to(hv.Image, ['x', 'y'], dynamic=True)
layout = (im * polys + dmap * hlines).opts(
    opts.Curve(width=400, framewise=True), 
    opts.Polygons(fill_alpha=0.2, line_color='white'), 
    opts.VLine(color='black',alpha=1.))


In [ ]:
print(layout)

In [ ]:
comp1 = im*polys;print(comp1)
comp2 = dmap * hlines; print(comp2)

In [ ]:
print(im); print(polys)

In [ ]:
print(dmap);print(hlines)

In [ ]:
print(comp2)
# layout


In [ ]:
print(dmap)

In [ ]:
## debug purpose
def get_debug_div(*args, **kwargs):
    content = f'<p> args: {str(args)} </p>'
    content += """
    <p> kwargs:  </p>
    <ul>
    """
    for k,v in kwargs.items():
        content += f'<li>{k}: {v}</li>'
    content += '</ul>'
    return hv.Div(content)

# Dynamically create the debug div with the stream of interest
# dmap_debug = hv.DynamicMap(get_debug_div, streams=[box_stream])

In [ ]:
varname = varnames[7]
region = 'EA'
xrd = xrd_ea
# Given region and varname
ds = gv.Dataset(xrd[varname], ['X','Y','time'], varname, crs=ccrs.PlateCarree())

In [ ]:
dmap = datashade(ds.to(gv.Image, ['X','Y'], varname, crs=ccrs.PlateCarree(), 
                       dynamic=True)
                )
# dmap = ds.to(gv.Image, ['X','Y'], varname, crs=ccrs.PlateCarree(), dynamic=True)

print(dmap)
dmap

In [ ]:
polys = gv.Polygons([], crs=ccrs.PlateCarree())
boxes_stream = BoxEdit(source=polys)

In [ ]:
# boxes_stream

# debug box for boxes stream
dmap_debug = hv.DynamicMap(get_debug_div, streams=[boxes_stream])

In [ ]:
def roi_curves(data):
    if not data or not any(len(d) for d in data.values()):
        return hv.NdOverlay({0: hv.Curve([], 'time', varname)})
#     pdb.set_trace()
    curves = {}
    data = zip(data['x0'], data['x1'], data['y0'], data['y1'])
    for i, (x0,x1,y0,y1) in enumerate(data):
        selection = ds.select(X=(x0,x1), Y=(y0,y1))
        curves[i] = hv.Curve(selection.aggregate('time', np.nanmean), 'time', varname)
#         pdb.set_trace()
    return hv.NdOverlay(curves)

                               

In [ ]:
dmap_roi_curves = hv.DynamicMap(roi_curves, streams=[boxes_stream])


In [ ]:
time_values = xrd.get_index('time')
# time_values = trange
empty_tplot = hv.Curve( 
    (time_values, np.empty(len(time_values))), 
    'time', varname).opts(line_alpha=0.)
vlines = hv.HoloMap({t: empty_tplot * hv.VLine(t) for t in time_values},
                   kdims='time')
vlines*dmap_roi_curves

In [ ]:
trange = list(map(pd.Timestamp, hvd_ea.range('time')))
trange

hvd_ea.range('time')

In [ ]:
layout = (
    dmap*polys 
    + dmap_roi_curves#*vlines
    + dmap_debug
).cols(2)
    
layout.opts(
    opts.Curve(framewise=True),
    opts.VLine(color='black'),
)

In [ ]:
dmap.kdims

In [ ]:
type(dmap.redim.values(time=[trange[0]]))


In [ ]:
dmap.select(time=trange[0])

In [ ]:
hv.VLine(t.to_datetime64())

In [ ]:
t = pd.Timestamp('2019-04-01')

In [ ]:
t


In [ ]:
t.to_datetime64()

In [ ]:
c = hv.Curve([],[])

In [ ]:
hvplot = renderer.get_plot(c)


In [ ]:
f = hvplot.state


In [ ]:
f.xaxis.

In [ ]:
c.print_param_values()

In [ ]:
hv.help(hv.Curve)

In [ ]:
from bokeh.models import DatetimeTickFormatter

In [ ]:
f.xaxis.formatter = DatetimeTickFormatter()

In [ ]:
display(c)
c.opts(opts.Curve(xformatter=DatetimeTickFormatter(hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )))

In [ ]:
f,#f.xaxis.formatter

In [ ]:
f.xaxis.formatter = DatetimeTickFormatter()

In [ ]:
f.xaxis.formatter